In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/CAPSTONE_DATASETS'

ALL_DOWN_SYNDROME_IMAGES  Deletion_syndrome  TEXTUAL_DATA
Autism_Images		  GMDB		     TRAIN1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler  # For mixed precision training

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define custom Dataset class to load grayscale images
class GrayscaleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("L")  # Convert to grayscale
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations (resize, normalize, and minimal augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Only horizontal flip for faster training
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize grayscale
])

# Load dataset paths and labels
dataset_path_down = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/AUTISTIC_GRAY_IMAGES"
dataset_path_healthy = "/content/drive/MyDrive/CAPSTONE_PROCESSED/TRAIN/NON_AUTISTIC_GRAY_IMAGES"
image_paths = []
labels = []

# Label 0: Autistic
for filename in os.listdir(dataset_path_down):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path_down, filename))
        labels.append(0)

# Label 1: Non-Autistic (Healthy)
for filename in os.listdir(dataset_path_healthy):
    if filename.endswith((".jpg", ".png")):
        image_paths.append(os.path.join(dataset_path_healthy, filename))
        labels.append(1)

print(f"Total image paths found: {len(image_paths)}")
print(f"Total labels found: {len(labels)}")

# Split into train and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

train_dataset = GrayscaleDataset(train_paths, train_labels, transform=transform)
val_dataset = GrayscaleDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)  # Increased batch size
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)  # Increased batch size

# Load VGG19 pretrained model
vgg19 = models.vgg19(pretrained=True)

# Modify the first conv layer to accept grayscale (1 channel)
vgg19.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)

# Modify the final fully connected layer for 2-class output
vgg19.classifier[6] = nn.Linear(vgg19.classifier[6].in_features, 2)

# Move model to GPU if available
vgg19.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(), lr=0.001)

# Learning Rate Scheduler (if necessary, helps stabilize training and can speed up convergence)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Mixed Precision Scaler
scaler = GradScaler()

# Training function with Mixed Precision
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with autocast():  # Mixed precision context
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()  # Scale loss for mixed precision
            scaler.step(optimizer)  # Step the optimizer
            scaler.update()  # Update the scaler

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

        # Update learning rate
        scheduler.step()

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_vgg19_model_autism.pth")
            print("Model saved!")

# Train the model
train_model(vgg19, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)


Total image paths found: 1265
Total labels found: 1265


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_We

Epoch [1/10], Loss: 8.9801, Accuracy: 51.48%
Validation Accuracy: 49.41%
Model saved!
Epoch [2/10], Loss: 0.8024, Accuracy: 51.38%
Validation Accuracy: 49.41%
Epoch [3/10], Loss: 0.7591, Accuracy: 50.79%
Validation Accuracy: 49.80%
Model saved!
Epoch [4/10], Loss: 0.6953, Accuracy: 52.17%
Validation Accuracy: 51.38%
Model saved!
Epoch [5/10], Loss: 0.7284, Accuracy: 50.30%
Validation Accuracy: 50.99%
Epoch [6/10], Loss: 0.6919, Accuracy: 54.94%
Validation Accuracy: 56.52%
Model saved!
Epoch [7/10], Loss: 0.6838, Accuracy: 53.46%
Validation Accuracy: 51.78%
Epoch [8/10], Loss: 0.6711, Accuracy: 59.88%
Validation Accuracy: 60.87%
Model saved!
Epoch [9/10], Loss: 0.6580, Accuracy: 58.60%
Validation Accuracy: 66.01%
Model saved!
Epoch [10/10], Loss: 0.6518, Accuracy: 60.47%
Validation Accuracy: 64.82%
